In [8]:
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import scipy.optimize 
from datetime import datetime,time,timedelta
from strategy2 import strategy2
from strategy1 import strategy1
from strategy3 import strategy3


In [53]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import floor

factor_returns = pd.read_csv("../../Analysis/factors_returns_multi.csv")
factors = pd.read_csv("../../Factors/merge_Factors.csv")
earning_factors = ["Momentum_1", "Momentum_2"]
risk_factors = ["Financial Quality", "Turnover"]
all_factors = earning_factors + risk_factors

def strategy2(date):
    seg_num = 5
    
    # get factors score from factors
    factors_score = factors[["Date", "Index"]+earning_factors+risk_factors]
    factors_score = factors_score.loc[factors_score["Date"]==date]
    for factor in all_factors:
        factors_score = factors_score.drop_duplicates(subset=[factor])
    for factor in all_factors:
        factors_score[factor+"_score"] = None
    num_row = factors_score.shape[0]
    cut_length = floor(num_row/seg_num)
    cutting_point = [i*cut_length for i in range(seg_num)]
    cutting_point.append(num_row)
    stock_index = []
    score_list = list(np.arange(0, 1, 1/seg_num)+1/seg_num)
    score_list.reverse()
    
    # get stock indexes in each region
    for i in range(len(cutting_point)-1):
        stock_index.append(list(range(cutting_point[i], cutting_point[i+1])))

    for factor in earning_factors:
        factors_score = factors_score.sort_values(by=factor, ascending=False)
        col_index = list(factors_score.columns).index(factor+"_score")
        for i in range(len(stock_index)):
            factors_score.iloc[stock_index[i], col_index] = 100*score_list[i]
    for factor in risk_factors:
        factors_score = factors_score.sort_values(by=factor, ascending=True)
        col_index = list(factors_score.columns).index(factor+"_score")
        for i in range(len(stock_index)):
            factors_score.iloc[stock_index[i], col_index] = 100*score_list[i]
    factors_score["total_score"] = factors_score[earning_factors[0]+"_score"] + factors_score[earning_factors[1]+"_score"] + factors_score[risk_factors[0]+"_score"] + factors_score[risk_factors[1]+"_score"]

    factors_score = factors_score.sort_values(by="total_score", ascending=False)
    selected_stocks = factors_score["Index"].head(10).tolist()
    return selected_stocks
    

In [54]:
factors = pd.read_csv("../../Factors/merge_Factors.csv")
factor_returns = pd.read_csv("../../Analysis/factors_returns_multi.csv")


In [55]:
date = "2022-10-03"
selected_stocks = strategy2(date)
factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
factor_returns = factor_returns.loc[factor_returns["Date"]==date]
factor_returns = factor_returns.iloc[:,1:]
stock_list = factors["Index"].tolist()
factors = factors.iloc[:, 3:]
factors = np.array(factors)
factor_returns = np.array(factor_returns)

In [56]:
stock_return = np.dot(factors, factor_returns.T).reshape(-1)
stock_list = np.array(stock_list)

In [57]:

from datetime import datetime 

In [58]:

def optimize(weights,tickers,start,end,calc_window,rf_rate,short=False,
             price_type='Close',clean='True'):
          
        '''
        weights; *type = list, initial guess weights
        
        start; input as str, data start date format 'YYYY-MM-DD' 
        end; input as str, data end date format 'YYYY-MM-DD'
        
        calc_window; input as str, calculation window for returns, 
        
        rf_rate; desired risk-free rate to benchmark against
        
        price_type; *input as str, default = 'Close', options = 'Adj Close', 
        'Close', 'High','Low','Open', 'Volume'
        
        short = bool, default = 'False', if true weight bounds -1 to 1
        
        clean; bool, default = 'False', if true, the function will drop NA rows
        '''
        start = datetime.strptime(start,'%Y-%m-%d')
        end = datetime.strptime(end,'%Y-%m-%d')
        
        prices = pdr.get_data_yahoo(tickers, start, end)[price_type]
        
        op = prices.pct_change(periods=int(calc_window))
        op.iloc[-1,:]=stock_return
        if clean == False:
            op
            
        else:
            op = op.dropna()
        
        vcv_matrix = np.cov(op.T)
        
        avg_return = op.mean()
        
        def sharpe(weights,vcv_matrix,avg_return):        
            
            var = np.dot(np.dot(weights,vcv_matrix),weights)
            
            ret =  np.dot(weights,avg_return)
            
            s_ratio = -(ret-rf_rate)/np.sqrt(var)
            
            return s_ratio
           
        
        sharpe_cons = ({'type':'eq','fun': lambda x: sum(x)-1})
        
        
        if short == False:
            sharpe_bnds = ((0,1),)*len(tickers)
        
        if short == True:
            sharpe_bnds = ((-1,1),)*len(tickers)
        
        optimal = scipy.optimize.minimize(sharpe,weights,bounds = sharpe_bnds,
                                 args = (vcv_matrix,avg_return),
                                 constraints = sharpe_cons)
        
        return optimal


In [59]:

weights=[0.1]*10
tickers=stock_list
#end为要预测仓位的那天
end='2022-10-03'
#start为end向前一个月
start='2022-09-03'
#不变，始终为1
calc_window='1'

In [60]:

# 确定无风险收益率
rf=pd.read_csv('daily-treasury-rates.csv',index_col=0)

#rf=rf.iloc[:,5]

rf.reset_index(inplace=True)

rf['Date']=[datetime.strptime(i,"%m/%d/%Y") for i in list(rf['Date'].values)]

start_time=datetime.strptime(start,"%Y-%m-%d")
end_time=datetime.strptime(end,"%Y-%m-%d")

rf=rf.set_index("Date")


for i in range(len(rf.index)):
    if list(rf.index)[i]-end_time<=timedelta(0):
        rfend_time=list(rf.index)[i]
        break
for j in range(len(rf.index)):
    if list(rf.index)[j]-start_time<=timedelta(0):
        rfstart_time=list(rf.index)[j]
        break

rf_rate=rf.loc[rfend_time:rfstart_time,'1 Yr'].mean()*0.01/360


In [61]:

optimize(weights,tickers,start,end,calc_window,rf_rate,short=True,price_type='Adj Close',clean='True')

     fun: -0.6969185710170434
     jac: array([0.06598385, 0.05546114, 0.06620257, 0.06658683, 0.0664262 ,
       0.13268272, 0.06597343, 0.06583884, 0.06620314, 0.06551199])
 message: 'Optimization terminated successfully'
    nfev: 165
     nit: 15
    njev: 15
  status: 0
 success: True
       x: array([ 0.85631409,  1.        ,  0.11602722,  0.0571925 ,  0.04715935,
       -1.        , -0.84509312, -0.08037373,  0.02679345,  0.82198024])

In [62]:
w=optimize(weights,tickers,start,end,calc_window,rf_rate,short=True,price_type='Adj Close',clean='True')
weight=list(w.x)

In [63]:
weight

[0.8563140881159422,
 1.0,
 0.11602722485749965,
 0.05719250291729601,
 0.04715934938248681,
 -1.0,
 -0.8450931167646714,
 -0.08037373456178747,
 0.02679344539772332,
 0.8219802406555108]